# Session 2.2: LLM-API-Interaktion, Kontext und Konversations-Historie

## Quellen

- OpenAI Developer Platform: https://platform.openai.com/docs/overview
- OpenAI API reference: https://platform.openai.com/docs/api-reference/introduction
- Ollama API Docs: https://github.com/ollama/ollama/blob/main/docs/api.md
- Ollama Model File: https://github.com/ollama/ollama/blob/main/docs/modelfile.md
- Temperature und Token-Probability: https://artefact2.github.io/llm-sampling/index.xhtml
- Explained: Tokens and Embeddings in LLMs (Medium) https://medium.com/the-research-nest/explained-tokens-and-embeddings-in-llms-69a16ba5db33
- Meta/Llama auf Huggingface: https://huggingface.co/meta-llama
- Deepseek Platform: https://platform.deepseek.com/usage


## Praxis

### 🛠️ Imports

In [ ]:
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, widgets
from aiworkshop_utils.openai_imports import OpenAI
from aiworkshop_utils import config

### 💡 Konzept: LLM-APIs und -SDKs

- Vor-/Nachteile, LLMs über Cloud oder on Premise ansprechen
- Firmen bieten LLM-Serving über APIs an - mit API-key ansprechbar, Verrechnung nach Tokens
- OpenAI API und SDK ist der Standard, weil sie die First-Mover sind
- Vorsicht: Verwechselung bei der Aussprache "OpenAI API" und "Open API" (Swagger)!

#### Screenshots

- Beispiel: Deepseek-Plattform Usage Stats

![Deepseek Usage](assets/image01_deepseek-platform-usage.png)

- Beispiel: Deepseek Models & Pricing

![Deepseek Platform Pricing](assets/image02_deepseek-platform-pricing.png)

- Beispiel: OpenAI Pricing

![Deepseek Platform Pricing](assets/image03_openai-platform-pricing.png)

#### Live-Beispiel Deepseek

In [ ]:
"""
client = OpenAI(api_key=config.DEEPSEEK_API_KEY, base_url=config.DEEPSEEK_API_URL)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Wie hat sich die Einwohnerzahl in Österreich entwickelt?"},
    ],
    stream=False
)

print(response.choices[0].message.content)
"""

### ⚡ OpenAI-SDK-Client erstellen mit lokaler Ollama API Base URL

In [3]:
openaisdk_client = OpenAI(
    base_url=config.OAPI_OPENAISDK_BASE_URL,
    api_key=config.OLLAMA_FAKE_API_KEY,  # fake key!
)

### ⚡ Erster Test einer lokalen OpenAI SDK Chat Completion mit Ollama API im Hintergrund

- Temperature und Token-Probability: https://artefact2.github.io/llm-sampling/index.xhtml
- neue API: Responses (https://platform.openai.com/docs/guides/text?api-mode=responses)
- alte API: Chat Completions (https://platform.openai.com/docs/guides/text?api-mode=chat)
- -> Ollama hat Responses noch nicht!
- Create chat completions Parameter: https://platform.openai.com/docs/api-reference/chat/create

In [4]:
response = openaisdk_client.chat.completions.create(
  model=config.OMODEL_LLAMA3D2, #config.OMODEL_DEEPSEEK
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke about a programmer."}
  ],
  temperature=0.7,
  top_p=1, # zwischen 0 und 1, bestimmt, aus welchen Token das Modell beim nächsten Schritt wählt; kumulative Wahrscheinlichkeit >= top_p
  stream=False,
  #max_tokens=10,
  #stop=None,
  #presence_penalty=0,
  #frequency_penalty=0,
  #n=3, -> funktioniert nicht in Ollama
  #logprobs=5 -> funktioniert nicht in Ollama
)

show_pretty_json(response)

```json
{
  "id": "chatcmpl-556",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Here's one:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs.",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1743886046,
  "model": "llama3.2:latest",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_ollama",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 39,
    "total_tokens": 56,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  }
}
```

### 🏋️ **[ÜBUNG_2.2.01]** Teste Single Completions mit unterschiedlichen Parametern

- *Diese Übung ist Teil von ÜBUNG3. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 3" bis zum 20.05.25 hoch.*

- TODOs:
    - Wechsle das Model aus
    - Probiere verschiedene User-Prompts aus
    - Probiere verschiedene Werte für die Parameter, speziell für Temperatur
- Fragen:
    - Was bedeuten die einzelnen Keys im Request und in der Response? (Tipp: https://platform.openai.com/docs/api-reference/chat/create) (muss nicht dokumentiert werden)
    - Welche Erfahrungen machst du?

### ⚡ Mehr Low-Level-Infos durch Interaktion mit Ollama-API

#### Ollama Endpunkte

```
---- COMPLETION ----
    A) Generate a completion       POST /api/generate
    B) Generate a chat completion  POST /api/chat

---- MODELS ----
    C) Create a Model              POST /api/create
    D) List Local Models           GET /api/tags
    E) Show Model Information      POST /api/show
    F) Copy a Model                POST /api/copy
    G) Delete a Model              DELETE /api/delete
    H) Pull a Model                POST /api/pull
    I) Push a Model                POST /api/push

---- EMBEDDINGS ----
    J) Generate Embeddings         POST /api/embed

---- ELSE ----
    K) List Running Models         GET /api/ps
    L) Version                     GET /api/version
```

###  ⚡ Ollama Endpunkt (K) 'List Running Models' GET /api/ps

In [ ]:
url = "http://localhost:11434/api/tags"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    print("Request failed with status code", response.status_code)

```
"name": "nomic-embed-text:latest",
"model": "nomic-embed-text:latest",
"modified_at": "2025-03-29T15:31:2...",
"size": 274302450,                           # in Bytes angegeben (ca. 2.02GB)
"digest": "0a109f422b47e3a......",           # digitaler Fingerabdruck
"details":
    "parent_model": "",
    "format": "gguf",                        # GGUF: "Grokking GGML Unified Format", optimiert für lokale Ausführung (ggml = leichtgw. ML-Lib)
    "family": "nomic-bert",
    "families": [
        "nomic-bert"
    ],
    "parameter_size": "137M",                # 137 Millionen Parameter (Weights and Biases)
    "quantization_level": "F16"
```

![Übersicht KI-Modell-Formate](assets/image05_übersicht-ki-modell-formate.png)

![Übersicht Quantisierungsstufen](assets/image06_quantisierungsstufen.png)

### ⚡ Ollama Endpunkt (A) 'Generate a completion' POST /api/generate: Einfache Completion

**Parameter**
- `model`: *(erforderlich)* Name des Modells  
- `prompt`: Eingabeaufforderung für die Generierung einer Antwort  
- `suffix`: Text, der nach der Modellantwort eingefügt wird (nicht bei llama3.1 -> LLaMA-based models (like llama3) are designed for sequential text generation rather than filling in missing text between a prompt and suffix.)
- `images` *(optional)*: Liste von base64-codierten Bildern (für multimodale Modelle wie LLaVA)  

**Erweiterte Parameter**
- `format`: Format der Antwort *(json oder JSON-Schema)*  
- `options`: Zusätzliche Modellparameter laut Modelfile-Dokumentation *(z. B. Temperatur)*  
- `system`: Systemnachricht *(überschreibt die im Modelfile definierte Nachricht)*  
- `template`: Vorlage für den Prompt *(überschreibt die im Modelfile definierte Vorlage)*  
- `stream`:  
  - **false**: Die Antwort wird als einzelnes Objekt zurückgegeben  
  - **true**: Antwort erfolgt als Stream  
- `raw`:  
  - **true**: Kein Formatierungsaufwand, nützlich für vollständig vorformulierte Prompts  
  - **false**: Standardformatierung wird angewendet  
- `keep_alive`:  
  - Zeitspanne, in der das Modell nach der Anfrage im Speicher bleibt *(Standard: 5 Minuten)*  
- `context` *(veraltet)*:  
  - Verwendet zur Aufrechterhaltung kurzer Konversationskontexte aus vorherigen Anfragen 

In [ ]:
# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAMA3D2
#prompt = "Give me one random word"
#prompt = '''Here is a question and an answer:
#    Question: How old are you?
#    Answer: I live in California.
#    Does the answer seem valid? Reply with 'Yes' or 'No' and explain briefly.'''
prompt = "Hi!"
stream_bool = False

# --------------
# execution
# --------------
# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "stream": stream_bool
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()
response_json_indented = json.dumps(response_json, indent=4, ensure_ascii=False)
response_text = response_json["response"]
tokens_per_second = response_json["eval_count"] / response_json["eval_duration"] * 10**9

# --------------
# output
# --------------
print(response_json_indented)
print("##### response #####")
print (response_text)
print(" ")
print("##### tokens per second #####")
print(tokens_per_second)

In [ ]:
print(response_json['context'])

### 🏋️ **[ÜBUNG_2.2.02]** Welchen Text ergeben diese Token IDs?

- *Diese Übung ist Teil von ÜBUNG3. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 3" bis zum 20.05.25 hoch.*
- Token-IDs:
    - [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 271, 128009, 128006, 882, 128007, 271, 19841, 0, 8574, 9857, 12, 31566, 48750, 304, 2991, 436, 19919, 74, 347, 17465, 0, 37674, 574, 82931, 295, 6754, 364, 68, 354, 6, 304, 364, 128009, 71090, 4060, 315, 12268, 304, 17495, 24277, 23124, 30, 128009, 128006, 78191, 128007, 271, 53545, 11, 10864, 9736, 13091, 71, 11, 15297, 3930, 2815, 9857, 12, 31566, 27348, 306, 34525, 83, 34143, 2268, 33717, 330, 68, 354, 1, 737, 2991, 55483, 7328, 330, 3812, 315, 12268, 498, 1101, 2761, 22855, 94483, 17495, 24277, 74095, 81, 29965, 13, 9419, 6127, 4466, 2395, 8510, 37907, 9857, 11, 6754, 75291, 15165, 11, 4543, 78968, 799, 13462, 11, 15297, 10021, 24277, 23124, 387, 55181, 6127, 2073, 32457, 31732, 268, 96917, 12666, 71877, 2781, 36708, 295, 12931, 382, 644, 89787, 13149, 12, 2073, 15883, 613, 424, 82212, 784, 615, 268, 15165, 47768, 9857, 75291, 11, 4543, 3453, 11246, 21740, 56013, 2761, 24277, 23124, 6529, 75361, 51691, 12778, 13, 9419, 6127, 13672, 4466, 330, 3812, 68, 1, 951, 21031, 652, 32673, 82, 11, 13582, 2486, 98244, 3744, 68, 3675, 76230, 11, 15297, 10112, 8969, 19028, 3276, 32251, 268, 12666, 8508, 268, 12931, 20649, 382, 41469, 305, 1885, 68, 11, 6754, 52427, 728, 5534, 31732, 0]
- Tokenizer Meta-Llama-3-8B-Instruct-tokenizer (2.3MB) über Moodle downloaden
- unzip
- Beispiel-Implementierung:
    ```
    token_ids = [...]
    tokenizer = AutoTokenizer.from_pretrained("/Users/michaelpfeiffer/notebooks_session2/Meta-Llama-3-8B-Instruct-tokenizer_folder/Unzipped/5f0b02c75b57c5855da9ae460ce51323ea669d8a")
    decoded_text = tokenizer.decode(token_ids, skip_special_tokens=False)
    ```
- **Gated Models auf Hugging Face, wie z.B. Llama**
    - Was sind Gated Models? Nur über Anmeldung auf Huggingface und Zustimmung der Nutzungsbedingungen beziehbar
    - Warum? Herausgeber, z.B. Meta für Llama3, möchten kontrollieren, wer Zugriff hat
    - Oft gilt Lizenz nur für Forschung, Nicht-kommerzielle-Nutzung
    - Damit akzeptiert man rechtliche Rahmenbedingungen

![Gated Huggingface Model](assets/image07_gated-huggingface-model.png)

### 🔓 **[LÖSUNG_2.2.02]** Welchen Text ergeben diese Token-IDs?

In [8]:
token_ids = [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 271, 128009, 128006, 882, 128007, 271, 19841, 0, 8574, 9857, 12, 31566, 48750, 304, 2991, 436, 19919, 74, 347, 17465, 0, 37674, 574, 82931, 295, 6754, 364, 68, 354, 6, 304, 364, 128009, 71090, 4060, 315, 12268, 304, 17495, 24277, 23124, 30, 128009, 128006, 78191, 128007, 271, 53545, 11, 10864, 9736, 13091, 71, 11, 15297, 3930, 2815, 9857, 12, 31566, 27348, 306, 34525, 83, 34143, 2268, 33717, 330, 68, 354, 1, 737, 2991, 55483, 7328, 330, 3812, 315, 12268, 498, 1101, 2761, 22855, 94483, 17495, 24277, 74095, 81, 29965, 13, 9419, 6127, 4466, 2395, 8510, 37907, 9857, 11, 6754, 75291, 15165, 11, 4543, 78968, 799, 13462, 11, 15297, 10021, 24277, 23124, 387, 55181, 6127, 2073, 32457, 31732, 268, 96917, 12666, 71877, 2781, 36708, 295, 12931, 382, 644, 89787, 13149, 12, 2073, 15883, 613, 424, 82212, 784, 615, 268, 15165, 47768, 9857, 75291, 11, 4543, 3453, 11246, 21740, 56013, 2761, 24277, 23124, 6529, 75361, 51691, 12778, 13, 9419, 6127, 13672, 4466, 330, 3812, 68, 1, 951, 21031, 652, 32673, 82, 11, 13582, 2486, 98244, 3744, 68, 3675, 76230, 11, 15297, 10112, 8969, 19028, 3276, 32251, 268, 12666, 8508, 268, 12931, 20649, 382, 41469, 305, 1885, 68, 11, 6754, 52427, 728, 5534, 31732, 0]
 
from transformers import AutoTokenizer
 
tokenizer = AutoTokenizer.from_pretrained(r"5f0b02c75b57c5855da9ae460ce51323ea669d8a")
decoded_text = tokenizer.decode(token_ids, skip_special_tokens=False)
print(decoded_text)

<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023

<|eot_id|><|start_header_id|>user<|end_header_id|>

Super! Die Token-IDs wurden in Text rückkodiert! Aber was bedeutet das 'eot' in '<|eot_id|>'? End of Turn in einer Konversation?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Ja, ich bin froh, dass du die Token-IDs wiederentdeckt hast!

Das "eot" im Text steht für "End of Turn", also der Abschluss einer Konversationsrunde. Es ist ein spezielles Token, das verwendet wird, um anzudeuten, dass eine Konversation beendet ist und keine weiteren Antworten oder Fragen erwartet werden.

In verschiedenen Chat- und Sprachverkehrssystemen wird dieses Token verwendet, um den Fortschritt der Konversation zu kennzeichnen. Es ist wie ein "Ende" des Gesprächs, bei dem beide Parteien wissen, dass sie nicht mehr antworten oder fragen werden können.

Ich hoffe, das hilft dir weiter!


- Versteckte Token-IDs: machen Struktur der Konversation klar, nicht sichtbar für Endnutzer:in, aber sehr wichtig für Modell
    - <|start_header_id|>
    - <|end_header_id|>
    - <|eot_id|> -> "end of turn"-ID, das Ende einer Eingabe, sei es vom System, vom User oder vom Assistant

In [ ]:
decoded_text_2 = tokenizer.decode([0, 1, 2, 3, 4, 5, 6, 7, 20000, 0, 30929], skip_special_tokens=False)
print(decoded_text_2)

In [ ]:
decoded_text_3 = tokenizer.decode([3404,12509,527,12738,0], skip_special_tokens=True)
print(decoded_text_3)

decoded_text_3 = tokenizer.decode([3404], skip_special_tokens=True)
print(decoded_text_3)
decoded_text_3 = tokenizer.decode([12509], skip_special_tokens=True)
print(decoded_text_3)
decoded_text_3 = tokenizer.decode([527], skip_special_tokens=True)
print(decoded_text_3)
decoded_text_3 = tokenizer.decode([12738], skip_special_tokens=True)
print(decoded_text_3)
decoded_text_3 = tokenizer.decode([0], skip_special_tokens=True)
print(decoded_text_3)

### 💡 Konzept: Kontext

- Die Liste "context" enthält die exakte Abfolge der Token, so wie sie vom Modell verarbeitet wurden.
- Bei einem Gespräch mit Verlauf enthält "context":
    - Den ersten Systemprompt
    - Frühere Nutzereingaben
    - Frühere Modellantworten
    - Die aktuelle Nutzereingabe
    - Die aktuelle Modellantwort

- Die ersten Token in "context" stammen vom Systemprompt.
- Neue Prompts und Antworten werden der Liste in Reihenfolge angehängt.
- Wenn es einen Gesprächsverlauf gibt, ist dieser vollständig in "context" enthalten – vor dem neuesten Prompt.

- Beispiel:
    - Nutzer: „Hallo, wie geht’s dir?“ → Tokenisierung: [1256, 890, 45, 289, 902]
    - → context: [1256, 890, 45, 289, 902]
    - Modell: „Mir geht’s gut!“ → Tokenisierung: [678, 45, 201, 90]
    - → context: [1256, 890, 45, 289, 902, 678, 45, 201, 90]

- Aber:
    - Es geht nicht nur um die reine Tokenisierung von Nutzereingabe und Modellantwort!
    - Es sind auch System- bzw. Meta-Token enthalten, wie z. B.:
        - < START >
        - < STOP >
        - < USER >
        - < ASSISTANT >
        - < CONTEXT >
    - Diese sorgen dafür, dass das Modell versteht, wer spricht, wo eine Antwort endet, und wie die Struktur des Gesprächs aussieht.

- Die Token-IDs unseres Kontextes, der über API mitgeschickt wurde, können wir mit einem Tokenizer wieder in Text rückverwandeln!
- Dafür müssen wir uns den Tokenizer holen - für Llama3.2 z.B. über HuggingFace - hier einloggen und Meta-Nutzerbestimmungen für Llama zustimmen

### 💡 Konzept: Structured Output

- durch Mitgabe eines Parameters wie dem Key 'format' oder 'response_format' und dem Value 'json' und einem Schema dazu wird Structured Output ermöglicht
- Wie funktioniert das und warum ist zu 100% sicher, dass die API-Response Syntax-korrekt in JSON ist?
    - 1. Request mit Parameter 'format'/'response_format'/... und Schema schicken
    - 2. Schema wird nicht als Teil des Prompts sichtbar, sondern im Hintergrund an Modell übergeben -> interner System-Prompt wird erzeugt "Yom must respond ONLY with a JSON object ..."
        - Hier kann sich das Modell noch immer daneben benehmen
    - 3. Ein Zwischensystem überprüft die Antwort - falls die nicht korrekt ist, wird ein Retry gemacht bzw. kann in manchen Fällen automatisch repariert/geparst werden
    - 4. Als Nutzer:in bekommt man ein 100% wohlgeformtes JSON
- ohne Structured Output und nur Anweisung in Prompt? Unsicher, oft mit z.B. "Hier ist dein JSON: ..." -> kann programmatisch nicht zu 100% sicher weitergegeben werden

### ⚡ Ollama Endpunkt (A) 'Generate a completion' POST /api/generate: Mit Structured Output

In [ ]:
# Generate a completion: structured output

# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAMA3D2
prompt = "Mein Name ist Rita Raster und ich bin Programmiererin. Ich bin 1995 geboren und liebe Musik machen. Ich habe nie Zeit. Antworte in JSON"
stream_bool = False
format_input = {
    "type": "object",
    "properties": {
        "Vorname": {"type": "string"},
        "Nachname": {"type": "string"},
        "Beruf": {"type": "string"},
        "Geburtsjahr": {"type": "integer"},
        "Hobbies": {"type": "array"},
        "Verfügbarkeit": {"type": "boolean"}
    },
    "required": [
        "Vorname",
        "Beruf",
        "Geburtsjahr",
        "Hobbies",
        "Verfügbarkeit"
    ]
  }

# --------------
# execution
# --------------
# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "stream": stream_bool,
    "format": format_input
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()

# --------------
# output
# --------------
# Print API-response:
#print(json.dumps(response_json, indent=4, ensure_ascii=False))

# Print only model-reponse:
# a) Extract the response as a string
response_text = response_json["response"]
# b) Convert the string back into a Python dictionary
response_data = json.loads(response_text)
# c) Print the formatted JSON response
print(json.dumps(response_data, indent=4, ensure_ascii=False))

### 🏋️ **[ÜBUNG_2.2.03]** Structured Output generieren, mit Pydantic statt pure JSON Schema

- *Diese Übung ist Teil von ÜBUNG3. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 3" bis zum 20.05.25 hoch.*
- Probiere verschiedene Prompts mit verschiedenen JSON-Schemas für Structured Output und bekomme ein Gefühl für Stärken/Schwächen
- Probiere es mit einer Pydantic Klasse als input -> der Format Parameter kann keine Pydantic-Klasse annehmen, aber es gäbe einen Weg
- Warum der Name "Pydantic"? "Py" von Python, "dentic" von "pedantic" - übertrieben detailverliebt und regelkonform auf Korrektheit und Präzsion achten -> Pydantic dient zur Datenvalidierung und Typisierung

### 🏋️ **[LÖSUNG_2.2.03]** Structured Output generieren, mit Pydantic statt pure JSON Schema

In [ ]:
# Generate a completion: structured output

# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAMA3D2
prompt = "Mein Name ist Rita Raster und ich bin Programmiererin. Ich bin 1995 geboren und liebe Musik machen. Ich habe nie Zeit. Antworte in JSON"
stream_bool = False
class PersonInput(BaseModel):
    Vorname: str
    Nachname: Optional[str] = None
    Beruf: str
    Geburtsjahr: int
    Hobbies: List[str]
    Verfügbarkeit: bool

# --------------
# execution
# --------------
# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "stream": stream_bool,
    "format": PersonInput.model_json_schema() # hier die Umwandlung auf JSON-Schema; mit OpenAI-SDK kann man direkt Pydantic-Klasse belassen
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()

# --------------
# output
# --------------
# Print API-response:
#print(json.dumps(response_json, indent=4, ensure_ascii=False))

# Print only model-reponse:
# a) Extract the response as a string
response_text = response_json["response"]
# b) Convert the string back into a Python dictionary
response_data = json.loads(response_text)
# c) Print the formatted JSON response
print(json.dumps(response_data, indent=4, ensure_ascii=False))

### 💡 Konzept: Multimodale Models

- Beispiel Text-Prompt mit Bild
    - prompt = "What can you see in the picture?"
    - images = [sunset.jpg]
    - ```[<vision_start>, [EMBEDDINGS_DES_BILDS], <vision_end>, <text_start>, "What can you see in the picture?", <text_end>]```
    - Ein visuelles Embedding wird in das Kontextfenster eingespeist, durch Vision-Encoder (z.B. CLIP-ViT, ResNet, Swin)

### ⚡ Ollama Endpunkt (A) 'Generate a completion' POST /api/generate: Mit Image als weiteren Input

In [ ]:
# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAVA
prompt = "What can you see in the picture?"
stream_bool = False
image_path = config.ASSETS_PATH + "sunset.jpg"
image_base64 = encode_image(image_path)

# --------------
# funcs
# --------------
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# --------------
# execution
# --------------
# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "stream": stream_bool,
    "images": [image_base64]
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()

# --------------
# output
# --------------
# Print API-response:
#print(json.dumps(response_json, indent=4, ensure_ascii=False))
# Print only model-reponse:
response_text = response_json["response"]
print(response_text)

### ⚡ Ollama Endpunkt (A) 'Generate a completion' POST /api/generate: Mit Temperatur-, Top-K- und Top-P-Unterschiede 

- temperature, top_p, top_k resource and tool for checking: https://artefact2.github.io/llm-sampling/index.xhtml
- temperature: 0-1 und darüber
- top_p: 0-1 (wo ist die Grenze des probibilistischen Durchschnitts pro Wort)
- top_k: 0-1000 (wieviele der Wörter sollen inkludiert sein)
    - Temperature: Ein Wert von 0,7 wird häufig verwendet, um ein ausgewogenes Verhältnis zwischen Kreativität und Kohärenz zu erreichen.
    - Top_k: Ein Wert von 50 wird oft gewählt, um die Auswahl auf die 50 wahrscheinlichsten Token zu beschränken und so die Vielfalt zu erhöhen. ￼
    - Top_p: Ein Wert von 0,9 bedeutet, dass die Auswahl der nächsten Token so erfolgt, dass die kumulative Wahrscheinlichkeit 90 % erreicht, was zu vielfältigeren und weniger repetitiven Antworten führt.


In [ ]:
# Generate a completion: options - temperature

# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAMA3D2
prompt = "Give me 5 random words."
#prompt = "How is apple cake made? Describe it in 6 sentences."
stream_bool = False
high_randomness = True

# --------------
# execution
# --------------
# randomness:
temperature_low_randomness = 0.01
top_p_low_randomness = 0.05
top_k_low_randomness = 1
temperature_high_randomness = 1.9
top_p_high_randomness = 0.99
top_k_high_randomness = 1000

if high_randomness == True:
    temperature = temperature_high_randomness
    top_p = top_p_high_randomness
    top_k = top_k_high_randomness
else:
    temperature = temperature_low_randomness
    top_p = top_p_low_randomness
    top_k = top_k_low_randomness

# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "stream": stream_bool,
    "options": {
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "seed": None
    },
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()

# --------------
# output
# --------------
# Print API-response:
#print(json.dumps(response_json, indent=4, ensure_ascii=False))

# Print only model-reponse:
print(response_json["response"])

### ⚡ Ollama Endpunkt (A) 'Generate a completion' POST /api/generate: System-Prompt anpassen

In [ ]:
# Generate a completion: system

# --------------
# vars
# --------------
endpoint = config.OAPI_GENERATE_URL
model = config.OMODEL_LLAMA3D2
prompt = "What is a train?"
#prompt = '''Here is a question and an answer:
#    Question: How old are you?
#    Answer: I live in California.
#    Does the answer seem valid? Reply with 'Yes' or 'No' and explain briefly.'''
system = "You are an old man and communicate in that way."
stream_bool = False

# --------------
# execution
# --------------
# Request Data:
data = {
    "model": model,
    "prompt": prompt,
    "system": system,
    "stream": stream_bool
}
# Send Request and Jsonify it, to be able to print it:
response = requests.post(endpoint, json=data)
response_json = response.json()

# --------------
# output
# --------------
print(response_json["response"])

### 🏋️ **[ÜBUNG_2.2.04]** System-Prompt-Beeinflussung

- *Diese Übung ist Teil von ÜBUNG3. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 3" bis zum 20.05.25 hoch.*
- Versuche verschiedene System-Prompts mit anschließenden Prompts und beobachte die LLM-Antworten
- Wenn du an realistische Use-Cases denkst - was könnte Sinn machen, in den System-Prompt zu schreiben?

### ⚡ Ollama Endpunkt (B) 'Generate a chat completion' POST /api/chat: Einfache Completion

**Parameter**
- `model`: (required) the model name
- `messages`: the messages of the chat, this can be used to keep a chat memory
- `tools`: list of tools in JSON for the model to use if supported

The message object has the following fields:
- `role`: the role of the message, either system, user, assistant, or tool
- `content`: the content of the message
- `images` (optional): a list of images to include in the message (for multimodal models such as llava)
- `tool_calls` (optional): a list of tools in JSON that the model wants to use

**Weitere Parameter**
- `format`: the format to return a response in. Format can be json or a JSON schema.
- `options`: additional model parameters listed in the documentation for the Modelfile such as temperature
- `stream`: if false the response will be returned as a single response object, rather than a stream of objects
- `keep_alive`: controls how long the model will stay loaded into memory following the request (default: 5m)

In [ ]:
# --------------
# vars
# --------------
endpoint = config.OAPI_CHAT_URL
model = config.OMODEL_LLAMA3D2
messages = ''
prompt1 = "Tell me a small joke."
prompt2 = "Explain shortly why it is funny."
stream_bool = False

# --------------
# funcs
# --------------
# Initialize chat history
chat_memory = []

def send_chat_message(user_input, stream=False):
    global chat_memory  # Keep memory of the conversation

    # Append user message to chat history
    chat_memory.append({"role": "user", "content": user_input})

    # Define the payload for the API request
    payload = {
        "model": model,
        "messages": chat_memory,
        "stream": stream  # Control streaming (True or False)
    }

    # Make the API request
    response = requests.post(endpoint, json=payload)

    # Parse the response
    if response.status_code == 200:
        response_json = response.json()
        assistant_message = response_json.get("message", {}).get("content", "")

        # Append assistant response to memory
        chat_memory.append({"role": "assistant", "content": assistant_message})

        return assistant_message
    else:
        return f"Error: {response.status_code}, {response.text}"

# --------------
# execution
# --------------
# Send a test message
response_1 = send_chat_message(prompt1)
#print("Assistant:", response_1)

# Continue conversation
response_2 = send_chat_message(prompt2)
#print("Assistant:", response_2)

# --------------
# output
# --------------
# Print conversation history
print("\nChat History:")
for msg in chat_memory:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

### 💡 Konzept: Konversations-Historie

- Nett, wenn der Kontext eine System-Message und eine User-Message hat und sich an alles vorhergehende erinnern kann
- Beispiel:
    ```
    conversation_history = [
        {"role": "system", "content": "You are a helpful assistant that loves programming jokes."},
        {"role": "user", "content": "Hello!"},
        {"role": "assistant", "content": "Hi there! How can I assist you today?"},
        {"role": "user", "content": "Tell me a programming joke."},
        {"role": "assistant", "content": "Why do programmers prefer dark mode? Because light attracts bugs! 🐛"},
        {"role": "user", "content": "Haha, that's a good one. Can you give me another?"},
        {"role": "assistant", "content": "Sure! Why do Java developers wear glasses? Because they don’t C#! 😄"}
    ]
    ```

### **[ÜBUNG_2.2.05]** Mini-Chat-Applikation mit Conversation History

- *Diese Übung ist Teil von ÜBUNG3. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 3" bis zum 20.05.25 hoch.*

TODO:
- Es soll möglich sein, hier im Notebook oder wo du auch immer willst nach Klick auf "Play" der Zelle mit dem lokalen LLM zu chatten
- Sich abwechselnde Ausgaben "You", "Assistant", "Token Usage", wobei die/der User:in bei "You" den Content (Prompt) eingeben kann
- LLM soll alles von System-Prompt bis hin zu aktuellen Completions alles in der conversation_history mitnehmen

Fragen:
- Welche Kontextfenster-Größe hat das benutzte Modell - i.e. wieviele Tokens passen maximal in die History?
- Was passiert, wenn das Limit des Kontextfensters erreicht wurde?

### **[LÖSUNG_2.2.05]** Notebook-Chat-Applikation mit Conversation History

Input zu TODOs:
- normalerweise funktioniert user_input = input("Please type something") -> direkt in Jupyter Notebook
- in VSC besser mit ipywidgets arbeiten

Antworten auf Fragen:
- Welche Kontextfenster-Größe hat das benutzte Modell - i.e. wieviele Tokens passen maximal in die History?
    - mit z.B. 'ollama show llama3.2:latest' sieht man den Parameter 'context length', in diesem Fall 131072.
- Was passiert, wenn das Limit des Kontextfensters erreicht wurde?
    - Beim Überschreiten der Kontextgrenze wird die History von vorne gekürzt, Ollama macht das z.B. automatisch
    - Es gibt smarte Kürzungstechniken, die man anwenden kann (z.B. zusammenfassen)



In [ ]:
# Insert custom CSS for styling messages
display(HTML('''
<style>
    .user-message {
        background-color: #e6f7ff;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 5px;
    }
    .assistant-message {
        background-color: #f0f0f0;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 5px;
    }
    .token-usage {
        background-color: #fff7e6;
        padding: 5px;
        border-radius: 5px;
        margin-bottom: 5px;
        font-size: 0.9em;
        color: #333;
    }
</style>
'''))

# Set up logging to file only
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
    handlers=[logging.FileHandler("llm_chat_log.txt")]
)

conversation_history = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# Function to call LLM and return reply + usage
def chat_with_llm(prompt):
    conversation_history.append({"role": "user", "content": prompt})
    
    model = "llama3.2:latest"
    response = openaisdk_client.chat.completions.create(
        model=model,
        messages=conversation_history
    )
    
    usage = response.usage
    if usage:
        logging.info(
            f"Token usage — Prompt: {usage.prompt_tokens}, "
            f"Completion: {usage.completion_tokens}, "
            f"Total: {usage.total_tokens}"
        )
        usage_text = (
            f"💡 Token usage — "
            f"Prompt: `<code>{usage.prompt_tokens}</code>`, "
            f"Completion: `<code>{usage.completion_tokens}</code>`, "
            f"Total: `<code>{usage.total_tokens}</code>`"
        )
    else:
        usage_text = "⚠️ No token usage info available."

    assistant_reply = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": assistant_reply})
    
    return assistant_reply, usage_text

# Widgets
input_box = widgets.Text(
    placeholder='Type your message',
    description='You:',
    disabled=False
)
send_button = widgets.Button(description="Send")
clear_button = widgets.Button(description="Clear")
status_label = widgets.Label(value="")
output_area = widgets.Output()

# Layout
input_controls = widgets.HBox([input_box, send_button, clear_button])
ui = widgets.VBox([output_area, status_label, input_controls])
display(ui)

# Button callbacks
def on_send_clicked(b):
    user_input = input_box.value.strip()
    if not user_input:
        return
    input_box.value = ''
    
    with output_area:
        display(HTML(f'<div class="user-message"><strong>You:</strong> {user_input}</div>'))
    
    if user_input.lower() == 'exit':
        with output_area:
            display(HTML('<div class="assistant-message"><strong>Goodbye!</strong></div>'))
        send_button.disabled = True
        input_box.disabled = True
        return
    
    status_label.value = "Generating answer..."
    assistant_reply, usage_text = chat_with_llm(user_input)
    status_label.value = ""
    
    with output_area:
        display(HTML(f'<div class="assistant-message"><strong>Assistant:</strong> {assistant_reply}</div>'))
        display(HTML(f'<div class="token-usage">{usage_text}</div>'))

def on_clear_clicked(b):
    output_area.clear_output()
    send_button.disabled = False
    input_box.disabled = False

send_button.on_click(on_send_clicked)
clear_button.on_click(on_clear_clicked)